In [ ]:
"""
okitamark
HuggingFace AutoModelForCausalLM Streaming
"""

## モデルとログファイルをセットする
MODEL_NAME = '/home/users/model/ArrowPro-7B-KUJIRA' ## モデルを指定
# LOG_FILE = __file__.replace('.py', '.log') ## デフォルトログファイル名, jupyterの時はNone
LOG_FILE = None ## ファイル名 or None


"""
Argument, Logging
"""
import argparse
import logging

## 引数の処理
## jupyterの場合コメント
# parser = argparse.ArgumentParser(description='OkitaMark')
# parser.add_argument('--model', type=str, default=MODEL_NAME)
# parser.add_argument('--log', type=str, default=LOG_FILE)
# args = parser.parse_args()
# MODEL_NAME = args.model
# LOG_FILE = args.log

## LOG_FILEが指定されていたら、ログファイルも出力する
if LOG_FILE is not None:
    handlers=[logging.FileHandler(LOG_FILE, mode='w'), logging.StreamHandler()]
else:
    handlers=[logging.StreamHandler()]

## Logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s', handlers=handlers) ## Message Only
LOG = logging.getLogger(__name__)


LOG.info('MODEL_NAME: ' + MODEL_NAME)
if LOG_FILE is not None: LOG.info('LOG_FILE:' + LOG_FILE)



"""
Base
"""
import os, time
import subprocess, argparse
import torch, gc
## Warning非表示
import warnings
warnings.simplefilter('ignore')

## Logging
# import logging
# logging.basicConfig(level=logging.DEBUG, format='%(message)s') ## Message Only
# LOG = logging.getLogger(__name__)

## Util ##
## メモリ関連
def show_memory():
  bytes = torch.cuda.max_memory_allocated()
  return 'GPU allocated memory: {mem:.2f} GiB'.format(mem=(bytes / 1024**3)) # GB

def free_memory():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()

## GPU Info
def gpu_info():
    gpucmd = 'nvidia-smi --query-gpu=name --format=csv,noheader'
    gpuinfo = subprocess.check_output(gpucmd, shell=True)
    return 'GPU device: ' + gpuinfo.decode()

def gpu_mem():
    gpucmd = 'nvidia-smi --query-gpu=utilization.gpu,utilization.memory,memory.used --format=csv,,noheader'
    gpuinfo = subprocess.check_output(gpucmd, shell=True)
    return 'GPU memory: ' + gpuinfo.decode()



"""
モデルの定義
Transformer
"""
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, TextStreamer

model_name = MODEL_NAME

## free GPU Memory
if 'model' in globals():
    del model
if 'tokenizer' in globals():
    del tokenizer
free_memory()


## init Model
model = AutoModelForCausalLM.from_pretrained(
        model_name,  ## モデル名,
        device_map="auto", ## auto, cpu, cuda:0
        torch_dtype="auto", ## torch.float16,torch.bfloat16
        trust_remote_code=True,
        # load_in_8bit=True, ## 8bit　bitsandbytes
    )


## init Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
        model_name,  ## モデル名
        trust_remote_code=True
    )


# ストリーミング表示
streamer = TextStreamer(
    tokenizer,
    skip_prompt=True, ## 入力文は返さない
    skip_special_tokens=True ## </s> などの特殊トークンも不要
)


## 推論
def predict(prompt, max_token=4095, temperature=0.0001, top_p=0.0001, title=''):
    free_memory() ## メモリ解放
    token_ids = tokenizer.encode(prompt, return_tensors="pt")
    n_token_input = len(token_ids[0])
    LOG.info('**'+title+'************************************************')
    LOG.info('**'+model_name)
    LOG.info('**'+gpu_info().rstrip('\n'))
    LOG.info('**'+gpu_mem().rstrip('\n'))
    LOG.info('**'+show_memory())
    LOG.info('**input:')
    LOG.info(prompt)
    LOG.info('**output:')  
    torch.manual_seed(0) ## 乱数初期化
    ##　推論開始
    stime = time.perf_counter() ## 計測開始
    output_ids = model.generate(
        input_ids=token_ids.to(model.device),
        max_new_tokens=max_token, ## 生成するトークンの最大数(プロンプトのトークンを含まない)
        temperature=temperature,
        top_p=top_p,
        # top_k=0,
        do_sample=True, ## Top-KやTop-p Samplingなどの戦略を有効にする
        # num_beams=1 ## Multinomial Sampling 反復のリスクが軽減される（確率分布に基づいて次のトークンをランダムに選択）
        # repetition_penalty=1 ## 繰り返しペナルティ、1.0はペナルティなし。モデルやユースケースに非常に敏感
        # no_repeat_ngram_size=3, ## 単語の繰り返しがなくなるが 同じ単語が使えなくなるので長文生成などでは使いにくい
        # pad_token_id=tokenizer.pad_token_id,
        # bos_token_id=tokenizer.bos_token_id,
        # eos_token_id=tokenizer.eos_token_id,
        ## ArrowPro
        pad_token_id=tokenizer.eos_token_id,
        ## llama3
        # pad_token_id=tokenizer.eos_token_id,
        # eos_token_id=[
        #     tokenizer.eos_token_id,
        #     tokenizer.convert_tokens_to_ids("<|eot_id|>") ## llama3
        # ],
        streamer=streamer, ## ストリーミング表示する場合


    )
    tm = time.perf_counter() - stime ## 計測終了
    n_token_output = len(output_ids[0][token_ids.size(1) :])

    ## No Streaming
    ## streamer=streamer をコメントすること
    # output = tokenizer.decode(
    #     output_ids[0][token_ids.size(1) :],
    #     skip_special_tokens=True
    # )
    # LOG.info(output)
    # End
            
    ## 計測結果
    LOG.info('**Result: %s, Time: %.2f, Input: %d, Output: %d, Total: %d, Token/sec: %.1f' % (title, tm, n_token_input, n_token_output, n_token_input+n_token_output, n_token_output/tm)) ## 終了時間, 出力文字数    
    LOG.info('**Result: %s, %s' % (title, gpu_mem().rstrip('\n')))
    LOG.info('**Result: %s, %s' % (title, show_memory()))
    LOG.info('\n\n')



"""
プロンプトテンプレート
モデルに該当したプロンプトを指定する
"""

####
## Vicuna1.5
def qa_vicuna(input,
      system='A chat between a human and an assistant.' ## システムプロンプト
      ):
    return """{s}
### Human: {i}
### Assistant: """.format(s=system, i=input)

####
## llama2, elayza, stablelm
def qa_llama2(input,
      system='あなたは誠実で優秀な日本人のアシスタントです' ## システムプロンプト
      ):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    return "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
        bos_token='<s>', ##tokenizer.bos_token,
        b_inst=B_INST,
        system=f"{B_SYS}{system}{E_SYS}",
        prompt=input,
        e_inst=E_INST,
    )

####
## llama3
def qa_llama3(input,
      system='あなたは日本語を話すAIアシスタントです。日本語で回答してください。you MUST write Japanese language.' ## システムプロンプト
      ):
    return """<|start_header_id|>system<|end_header_id|>\n\n{s}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{i}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n""".format(s=system, i=input)

####
## 東工大LLM/swallow
def qa_swallow_chat(input,
      system='以下に、あるタスクを説明する指示があります。リクエストを適切に完了するための回答を記述してください。' ## システムプロンプト
      ):
    return """{s}
    
### 指示:
{i}

### 応答:
""".format(s=system, i=input)

####
## Gemma
def qa_gemma(input,
      system='' ## システムプロンプト
      ):
    return """<start_of_turn>user
{i}<end_of_turn>
""".format(s=system, i=input)

####
## ArrowPro
## pad_token_id=tokenizer.eos_token_id
def qa_arrowpro(user_query):
    sys_msg = "あなたは日本語を話す優秀なアシスタントです。回答には必ず日本語で答えてください。"
    template = """[INST] <<SYS>>
{}
<</SYS>>

{}[/INST]"""
    return template.format(sys_msg,user_query)
    

## set Prompt Template
qa = qa_arrowpro ## モデルに該当したプロンプトを指定する

## for Test
predict(qa("""自己紹介をしてください。"""), title='自己紹介')


LOG.info('fin.')